In [1]:
from dotenv import load_dotenv
load_dotenv('/Users/eshantdas/Desktop/YOutube_scripts/myenv/.env')

True

In [2]:

from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

model=ChatGroq( model="llama-3.1-70b-versatile")


In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field


In [4]:
class Tagging(BaseModel):
    """Put the sentiment and language in the form of a dictionary"""
    sentiment:str=Field(description="sentiment of the text which should be 'pos','neg' or 'neutral")
    language:str=Field(description="language of the text")

In [5]:
prompt=ChatPromptTemplate.from_messages([
    ("system","Think carefully and then tag the text as instrucred"),
    ("user","{input}")
]
)

In [6]:
# chain=(prompt | model.bind_tools(
#     tools=[Tagging],tool_choice='Tagging').with_config(run_name='Tagging')
#     | PydanticToolsParser(tools=[Tagging])
#     )

# chain.invoke({"input":"I love Eshant"})

In [7]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

from langchain_core.utils.function_calling import convert_to_openai_function

In [8]:
convert_to_openai_function(Tagging)

{'name': 'Tagging',
 'description': 'Put the sentiment and language in the form of a dictionary',
 'parameters': {'type': 'object',
  'properties': {'sentiment': {'description': "sentiment of the text which should be 'pos','neg' or 'neutral",
    'type': 'string'},
   'language': {'description': 'language of the text', 'type': 'string'}},
  'required': ['sentiment', 'language']}}

In [9]:
import json
def output_parser_letsgo(data):
    a=data
    b=a.additional_kwargs['function_call']['arguments']
    c=json.loads(b)
    return c
    
    

In [10]:
import json
def output_parser_letsgo(data):
    a=data
    b=a.additional_kwargs['function_call']['arguments']
    c=json.loads(b)
    return c
    
    


from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda
parser=JsonOutputParser(pydantic_object=Tagging)

tagging_function=[convert_to_openai_function(Tagging)]
model_with_function=model.bind(
    functions=tagging_function,
    function_call={"name":"Tagging"}
)

tagging_chain=prompt | model_with_function | RunnableLambda(output_parser_letsgo)

In [11]:
a=tagging_chain.invoke({"input":"i love langchain"})
a

{'language': 'English', 'sentiment': 'pos'}

## `Extraction`

In [12]:
from typing import Optional,List
class Person(BaseModel):
    """Information about a person"""
    name:str=Field(description="persons name")
    age:Optional[int]=Field(description="person's age")
    hobby:str=Field(description="person hobby")

In [13]:
class Information(BaseModel):
    """Information to extract"""
    person:List[Person]=Field(description="List of info about people")

In [14]:
extraction_function=[convert_to_openai_function(Information)]
extraction_model=model.bind(
    functions=extraction_function,
    function_call={"name":"Information"}
)

In [15]:
import json
def output_parser_letsgo(data):
    a=data
    b=a.additional_kwargs['function_call']['arguments']
    c=json.loads(b)
    return c['person']

In [16]:
extraction_chain=prompt | extraction_model | RunnableLambda(output_parser_letsgo)

extraction_chain.invoke("Eshant is 20 years old and he likes to play Cricket and his mom whose name is Barna lobves to cook chicken. He also has a brother named Riyan who is 20 years old but he likes to play football")

[{'name': 'Eshant', 'hobby': 'Cricket', 'age': 20},
 {'name': 'Barna', 'hobby': 'cooking chicken'},
 {'name': 'Riyan', 'hobby': 'football', 'age': 20}]